# Diabetes Patients Early Readmissions Prediction

**Authors:** [Peter Macinec](https://github.com/pmacinec), [Frantisek Sefcik](https://github.com/FrantisekSefcik)

## Evaluation

In this jupyter notebook, we evaluate our model for diabetes patients early readmission prediction.

### Setup and import libraries